In [1]:
import pandas as pd
import numpy as np

import requests
import openpyxl as xl
import time
import json

import folium
import re

In [62]:
df2 = pd.read_excel("최종_행정동별_상권변화지표.xlsx")
df2["상권변화지수"]=""
df2.head(3)

,지역구,행정동_ID,행정동,상권변화지표,Unnamed: 4,상권변화지수
0,송파구,11710631,가락1동,"HH,HH,HH,HH",0.0,
1,송파구,11710632,가락2동,"HL,HL,HL,HL",10.0,
2,송파구,11710620,가락본동,"LL,LL,LL,LL",NaN,


In [63]:
df2.shape

(424, 6)

In [64]:
semi = pd.read_excel("상권지수_수정.xlsx")
semi = semi[["통합","상권_변화_지표"]]
semi = semi.rename(columns={"상권_변화_지표":"상권변화지표"})
semi.head()

,통합,상권변화지표
0,0,"HH,HH,HH,HH"
1,10,"HL,HL,HL,HL"
2,30,"LL,LL,LL,LL"
3,28,"HL,LL,LL,LL"
4,26,"LL,LL,LL,LH"


In [65]:
for i in range(424):
    for j in range(28):
        if df2.상권변화지표[i] == semi.상권변화지표[j]:
            df2.상권변화지수[i] = semi.통합[j]

C:\Users\ajm10\AppData\Local\Temp\ipykernel_6540\1904072364.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.상권변화지수[i] = semi.통합[j]


In [66]:
dff=df2.copy()
dff=dff.drop(columns={"Unnamed: 4","상권변화지표"})
dff

,지역구,행정동_ID,행정동,상권변화지수
0,송파구,11710631,가락1동,0
1,송파구,11710632,가락2동,10
2,송파구,11710620,가락본동,30
3,구로구,11530595,가리봉동,10
4,금천구,11545510,가산동,30
...,...,...,...,...
419,용산구,11170580,효창동,0
420,용산구,11170510,후암동,20
421,동대문구,11230720,휘경1동,10
422,동대문구,11230730,휘경2동,29


In [69]:
dff.to_excel("최종_행정동별_상권변화지수.xlsx", encoding="utf-8")

# df1~df6 까지 확인

In [4]:
df1 = pd.read_excel("최종_제로마켓_위치.xlsx")
df1.head(3)

,상호명,도로명 주소,지역구,y,x,행정동
0,1.5도씨,서울특별시 관악구 조원로18길 15,관악구,37.483579,126.912646,조원동
1,Dear.eco(디어 에코),서울특별시 광진구 아차산로65길 22,광진구,37.541265,127.093793,구의3동
2,감탄상회,서울특별시 동작구 양녕로29길 16 101호,동작구,37.499339,126.943174,상도4동


In [11]:
df1.shape

(46, 6)

In [72]:
df2 = pd.read_excel("최종_행정동별_상권변화지수.xlsx")
df2.head(3)

,지역구,행정동_ID,행정동,상권변화지수
0,송파구,11710631,가락1동,0
1,송파구,11710632,가락2동,10
2,송파구,11710620,가락본동,30


In [73]:
df2.shape

(424, 4)

In [8]:
df3 = pd.read_excel("최종_호선명_역명_2020합계.xlsx")
df3.head(3)

,line_nm,station_nm,sum_line_station,지역구,getoff_sum,x,y,행정동
0,우이신설선,4.19민주묘지,688568,강북구,25855961,127.013746,37.649593,우이동
1,"3호선,8호선",가락시장,4401586,송파구,72306079,127.118234,37.492522,가락1동
2,7호선,가산디지털단지,8574201,금천구,18515469,126.882343,37.481072,가산동


In [13]:
df3.shape

(283, 8)

In [9]:
df4 = pd.read_csv("최종_고령층제외1인가구_주소별_가중치.csv", encoding="utf-8")
df4.head(3)

,지역구,행정동,전체인구,고령층 제외 1인가구
0,종로구,사직동,"3,839",1207
1,종로구,삼청동,"1,031",308
2,종로구,부암동,"3,773",916


In [14]:
df4.shape

(424, 4)

In [3]:
df5 = pd.read_excel("최종_서울시_대학_DB정보.xlsx")
df5.head(3)

,학교명,지역구,행정동
0,한영신학대학교,구로구,개봉1동
1,동양미래대학교,구로구,고척1동
2,삼육대학교,노원구,공릉2동


In [4]:
df5.shape

(59, 3)

In [2]:
df6 = pd.read_excel("최종_자치구별_1인가구_배달주문_및_총합.xlsx")
df6.head(3)

,시군구코드,지역구,배달주문 빈도수,전체인구,고령층제외_1인가구,1인가구 평균 배달주문 빈도수
0,11110,종로구,25983,63414,21086,1.232239
1,11140,중구,22818,55093,18292,1.247431
2,11170,용산구,36881,95714,30486,1.209768


In [22]:
df6.shape

(25, 6)

## df 병합

In [234]:
# 상권변화지수 + 1인가구 
df = pd.read_excel("통합기준.xlsx")
df

,지역구,행정동_ID,행정동,상권변화지수,전체인구,고령층 제외 1인가구
0,송파구,11710631,가락1동,0,9534,1045
1,송파구,11710632,가락2동,10,11482,1892
2,송파구,11710620,가락본동,30,10576,2642
3,구로구,11530595,가리봉동,10,4901,2244
4,금천구,11545510,가산동,30,14124,8016
...,...,...,...,...,...,...
419,용산구,11170580,효창동,0,3922,894
420,용산구,11170510,후암동,20,7283,2314
421,동대문구,11230720,휘경1동,10,6643,2730
422,동대문구,11230730,휘경2동,29,10915,3915


In [235]:
table=df.copy()
table

,지역구,행정동_ID,행정동,상권변화지수,전체인구,고령층 제외 1인가구
0,송파구,11710631,가락1동,0,9534,1045
1,송파구,11710632,가락2동,10,11482,1892
2,송파구,11710620,가락본동,30,10576,2642
3,구로구,11530595,가리봉동,10,4901,2244
4,금천구,11545510,가산동,30,14124,8016
...,...,...,...,...,...,...
419,용산구,11170580,효창동,0,3922,894
420,용산구,11170510,후암동,20,7283,2314
421,동대문구,11230720,휘경1동,10,6643,2730
422,동대문구,11230730,휘경2동,29,10915,3915


In [236]:
# 행정동별 제로마켓 개수
trans = df1.copy()
trans = trans.groupby ("행정동").count()
zero = trans[["상호명"]]
zero = zero.rename(columns={"상호명":"제로마켓개수"})
zero = zero.reset_index()
zero

,행정동,제로마켓개수
0,가락본동,1
1,가회동,1
2,구의3동,1
3,길음1동,1
4,남가좌1동,1
5,남가좌2동,1
6,남영동,1
7,남현동,1
8,녹번동,1
9,당산1동,1


In [251]:
## 1인가구 데이터 + 상권변화지수 + 제로마켓개수

table["제로마켓개수"] = ""

for i in range(424):
    for j in range(43):
        if table["행정동"][i] == zero["행정동"][j]:
            table["제로마켓개수"][i] = zero["제로마켓개수"][j]
    #    else: 
     #       table["제로마켓개수"][i] = 0
            

C:\Users\ajm10\AppData\Local\Temp\ipykernel_6540\3339282588.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table["제로마켓개수"][i] = zero["제로마켓개수"][j]


In [252]:
table["제로마켓개수"][(table["제로마켓개수"]!=1)&(table["제로마켓개수"]!=2)] = 0
table_z = table.copy()
table_z.head(5)

C:\Users\ajm10\AppData\Local\Temp\ipykernel_6540\4009045861.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table["제로마켓개수"][(table["제로마켓개수"]!=1)&(table["제로마켓개수"]!=2)] = 0


,지역구,행정동_ID,행정동,상권변화지수,전체인구,고령층 제외 1인가구,제로마켓개수
0,송파구,11710631,가락1동,0,9534,1045,0
1,송파구,11710632,가락2동,10,11482,1892,0
2,송파구,11710620,가락본동,30,10576,2642,1
3,구로구,11530595,가리봉동,10,4901,2244,0
4,금천구,11545510,가산동,30,14124,8016,0


In [253]:
# 행정동별 1인가구 데이터 + 상권변화지수 + 제로마켓개수 + 대학교 개수
trans = df5.copy()
trans = trans.groupby ("행정동").count()
univ = trans[["학교명"]]
univ = univ.rename(columns={"학교명":"학교개수"})
univ = univ.reset_index()
univ

,행정동,학교개수
0,개봉1동,1
1,고척1동,1
2,공릉2동,3
3,광장동,1
4,군자동,1
5,낙성대동,1
6,남가좌2동,1
7,논현2동,1
8,대흥동,1
9,도화동,1


In [254]:
table_z["대학교개수"]=""

for i in range(424):
    for j in range(53):
        if table_z["행정동"][i] == univ["행정동"][j]:
            table_z["대학교개수"][i] = univ["학교개수"][j]
        else: 
            table_z["대학교개수"][i] = 0


C:\Users\ajm10\AppData\Local\Temp\ipykernel_6540\795215163.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table_z["대학교개수"][i] = 0
C:\Users\ajm10\AppData\Local\Temp\ipykernel_6540\795215163.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table_z["대학교개수"][i] = univ["학교개수"][j]


In [255]:
table_z.head()

,지역구,행정동_ID,행정동,상권변화지수,전체인구,고령층 제외 1인가구,제로마켓개수,대학교개수
0,송파구,11710631,가락1동,0,9534,1045,0,0
1,송파구,11710632,가락2동,10,11482,1892,0,0
2,송파구,11710620,가락본동,30,10576,2642,1,0
3,구로구,11530595,가리봉동,10,4901,2244,0,0
4,금천구,11545510,가산동,30,14124,8016,0,0


In [256]:
table_z=table_z.rename(columns={"제로마켓개수":"count_zero","대학교개수":"count_univ"})
table_u=table_z.copy()
table_u=table_u.rename(columns={"고령층 제외 1인가구":"1인가구"})
table_u.head()

,지역구,행정동_ID,행정동,상권변화지수,전체인구,1인가구,count_zero,count_univ
0,송파구,11710631,가락1동,0,9534,1045,0,0
1,송파구,11710632,가락2동,10,11482,1892,0,0
2,송파구,11710620,가락본동,30,10576,2642,1,0
3,구로구,11530595,가리봉동,10,4901,2244,0,0
4,금천구,11545510,가산동,30,14124,8016,0,0


### 행정동별 1인가구 데이터 + 상권변화지수 + 제로마켓개수 + 대학교 개수 + 행정동별 지하철역개수 

In [257]:
trans = df3.copy()
trans = trans.groupby ("행정동").count()
trans = trans[["station_nm"]]
trans = trans.rename(columns={"station_nm":"count_station"})
trans = trans.reset_index()
trans

,행정동,count_station
0,가락1동,2
1,가락2동,2
2,가산동,2
3,가양1동,2
4,갈현1동,1
...,...,...
202,화양동,1
203,회현동,2
204,효창동,1
205,휘경1동,1


In [258]:
table_u["count_station"]=""

for i in range(424):
    for j in range(207):
        if table_u["행정동"][i] == trans["행정동"][j]:
            table_u["count_station"][i] = trans["count_station"][j]
#        else: 
 #           table2["count_station"][i] = 0

C:\Users\ajm10\AppData\Local\Temp\ipykernel_6540\145153557.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table_u["count_station"][i] = trans["count_station"][j]


In [259]:
table_sc = table_u.copy()
table_sc["count_station"][(table_sc["count_station"]!=1)&(table_sc["count_station"]!=2)
      &(table_sc["count_station"]!=3)&(table_sc["count_station"]!=4)] = 0
table_sc

C:\Users\ajm10\AppData\Local\Temp\ipykernel_6540\710372798.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table_sc["count_station"][(table_sc["count_station"]!=1)&(table_sc["count_station"]!=2)


,지역구,행정동_ID,행정동,상권변화지수,전체인구,1인가구,count_zero,count_univ,count_station
0,송파구,11710631,가락1동,0,9534,1045,0,0,2
1,송파구,11710632,가락2동,10,11482,1892,0,0,2
2,송파구,11710620,가락본동,30,10576,2642,1,0,0
3,구로구,11530595,가리봉동,10,4901,2244,0,0,0
4,금천구,11545510,가산동,30,14124,8016,0,0,2
...,...,...,...,...,...,...,...,...,...
419,용산구,11170580,효창동,0,3922,894,0,0,1
420,용산구,11170510,후암동,20,7283,2314,0,0,0
421,동대문구,11230720,휘경1동,10,6643,2730,0,0,1
422,동대문구,11230730,휘경2동,29,10915,3915,0,0,0


### 행정동별 하차 승객수 합산 추가

In [260]:
trans = df3.copy()
trans["sum_getoff"] = trans.groupby(by = ['행정동'])["sum_line_station"].transform(sum)

trans=trans[["행정동","sum_getoff"]].drop_duplicates()
trans=trans.sort_values(["행정동"], ascending=True)
trans = trans.reset_index()
trans= trans.drop(columns={"index"})
trans

,행정동,sum_getoff
0,가락1동,6434890
1,가락2동,3449427
2,가산동,14509425
3,가양1동,4184708
4,갈현1동,8448654
...,...,...
202,화양동,13527276
203,회현동,15606508
204,효창동,1545629
205,휘경1동,9501588


In [261]:
table_sc["sum_getoff"]=""

In [262]:
for i in range(424):
    for j in range(207):
        if table_sc["행정동"][i] == trans["행정동"][j]:
            table_sc["sum_getoff"][i] = trans["sum_getoff"][j]

C:\Users\ajm10\AppData\Local\Temp\ipykernel_6540\1198349417.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table_sc["sum_getoff"][i] = trans["sum_getoff"][j]


In [265]:
table_sp=table_sc.copy()
table_sp["sum_getoff"][table_sp["sum_getoff"]==""]=0

C:\Users\ajm10\AppData\Local\Temp\ipykernel_6540\410061288.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table_sp["sum_getoff"][table_sp["sum_getoff"]==""]=0


In [266]:
table_sp

,지역구,행정동_ID,행정동,상권변화지수,전체인구,1인가구,count_zero,count_univ,count_station,sum_getoff
0,송파구,11710631,가락1동,0,9534,1045,0,0,2,6434890
1,송파구,11710632,가락2동,10,11482,1892,0,0,2,3449427
2,송파구,11710620,가락본동,30,10576,2642,1,0,0,0
3,구로구,11530595,가리봉동,10,4901,2244,0,0,0,0
4,금천구,11545510,가산동,30,14124,8016,0,0,2,14509425
...,...,...,...,...,...,...,...,...,...,...
419,용산구,11170580,효창동,0,3922,894,0,0,1,1545629
420,용산구,11170510,후암동,20,7283,2314,0,0,0,0
421,동대문구,11230720,휘경1동,10,6643,2730,0,0,1,9501588
422,동대문구,11230730,휘경2동,29,10915,3915,0,0,0,0


# 구 데이터 추가
### 지역구별 쓰레기 하루 배출량

In [277]:
result = table_sp.copy()

In [279]:
goo1 = pd.read_excel("최종_지역구별_쓰레기배출량.xlsx")
goo1 = goo1.iloc[1:]
goo1 = goo1.reset_index()
goo1 = goo1[["지역구","하루 배출량"]] # 지역구별 쓰레기 배출량
goo1.shape

(25, 2)

In [282]:
result["daily_waste"]=""

for i in range(424):
    for j in range(25):
        if result["지역구"][i] == goo1["지역구"][j]:
            result["daily_waste"][i] = goo1["하루 배출량"][j]

C:\Users\ajm10\AppData\Local\Temp\ipykernel_6540\4137038440.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result["daily_waste"][i] = goo1["하루 배출량"][j]


In [285]:
result2=result.copy()

### 지역구별 1인가구 배달주문 빈도수 (1년 기준 365일 평균)

In [286]:
goo2 = df6.copy()
goo2.head()

,시군구코드,지역구,배달주문 빈도수,전체인구,고령층제외_1인가구,1인가구 평균 배달주문 빈도수
0,11110,종로구,25983,63414,21086,1.232239
1,11140,중구,22818,55093,18292,1.247431
2,11170,용산구,36881,95714,30486,1.209768
3,11200,성동구,42585,122186,35224,1.208977
4,11215,광진구,62301,152090,54101,1.151568


In [288]:
result2["alone_order"]=""

for i in range(424):
    for j in range(25):
        if result2["지역구"][i] == goo2["지역구"][j]:
            result2["alone_order"][i] = goo2["1인가구 평균 배달주문 빈도수"][j]

C:\Users\ajm10\AppData\Local\Temp\ipykernel_6540\3378488005.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result2["alone_order"][i] = goo2["1인가구 평균 배달주문 빈도수"][j]


In [289]:
result2

,지역구,행정동_ID,행정동,상권변화지수,전체인구,1인가구,count_zero,count_univ,count_station,sum_getoff,daily_waste,alone_order
0,송파구,11710631,가락1동,0,9534,1045,0,0,2,6434890,4271.9,1.200815
1,송파구,11710632,가락2동,10,11482,1892,0,0,2,3449427,4271.9,1.200815
2,송파구,11710620,가락본동,30,10576,2642,1,0,0,0,4271.9,1.200815
3,구로구,11530595,가리봉동,10,4901,2244,0,0,0,0,1278.5,1.269451
4,금천구,11545510,가산동,30,14124,8016,0,0,2,14509425,3794.8,1.219473
...,...,...,...,...,...,...,...,...,...,...,...,...
419,용산구,11170580,효창동,0,3922,894,0,0,1,1545629,1052.6,1.209768
420,용산구,11170510,후암동,20,7283,2314,0,0,0,0,1052.6,1.209768
421,동대문구,11230720,휘경1동,10,6643,2730,0,0,1,9501588,3237.0,1.222093
422,동대문구,11230730,휘경2동,29,10915,3915,0,0,0,0,3237.0,1.222093


# 위도, 경도 좌표계 추가

In [304]:
xy = pd.read_excel("행정_법정동 중심좌표.xlsx")
xy = xy.rename(columns={"시군구":'지역구',"읍면동":"행정동"})
xy = xy.sort_values(["행정동"], ascending=True)
xy = xy.reset_index()
xy = xy[["위도", "경도","행정동"]]
xy

,위도,경도,행정동
0,37.496537,127.105726,가락1동
1,37.498639,127.126488,가락2동
2,37.495585,127.121830,가락본동
3,37.482704,126.888019,가리봉동
4,37.476681,126.883767,가산동
...,...,...,...
419,37.542828,126.960951,효창동
420,37.549901,126.980613,후암동
421,37.574428,127.039712,휘경1동
422,37.574428,127.039712,휘경2동


In [305]:
ss = pd.concat([result2, xy], axis=1)

In [306]:
ss.isnull().sum()

지역구              0
행정동_ID           0
행정동              0
상권변화지수           0
전체인구             0
1인가구             0
count_zero       0
count_univ       0
count_station    0
sum_getoff       0
daily_waste      0
alone_order      0
위도               0
경도               0
행정동              0
dtype: int64

In [313]:
ss = ss.iloc[:,:14]
ss

,지역구,행정동_ID,행정동,상권변화지수,전체인구,1인가구,count_zero,count_univ,count_station,sum_getoff,daily_waste,alone_order,위도,경도
0,송파구,11710631,가락1동,0,9534,1045,0,0,2,6434890,4271.9,1.200815,37.496537,127.105726
1,송파구,11710632,가락2동,10,11482,1892,0,0,2,3449427,4271.9,1.200815,37.498639,127.126488
2,송파구,11710620,가락본동,30,10576,2642,1,0,0,0,4271.9,1.200815,37.495585,127.121830
3,구로구,11530595,가리봉동,10,4901,2244,0,0,0,0,1278.5,1.269451,37.482704,126.888019
4,금천구,11545510,가산동,30,14124,8016,0,0,2,14509425,3794.8,1.219473,37.476681,126.883767
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,용산구,11170580,효창동,0,3922,894,0,0,1,1545629,1052.6,1.209768,37.542828,126.960951
420,용산구,11170510,후암동,20,7283,2314,0,0,0,0,1052.6,1.209768,37.549901,126.980613
421,동대문구,11230720,휘경1동,10,6643,2730,0,0,1,9501588,3237.0,1.222093,37.574428,127.039712
422,동대문구,11230730,휘경2동,29,10915,3915,0,0,0,0,3237.0,1.222093,37.574428,127.039712


In [314]:
ss.to_excel("수정_1027_df.xlsx")

In [5]:
# 행정동별 1인가구 데이터 + 상권변화지수 + 제로마켓개수 + 대학교 개수
trans = df5.copy()
trans = trans.groupby ("행정동").count()
univ = trans[["학교명"]]
univ = univ.rename(columns={"학교명":"학교개수"})
univ = univ.reset_index()
univ

,행정동,학교개수
0,개봉1동,1
1,고척1동,1
2,공릉2동,3
3,광장동,1
4,군자동,1
5,낙성대동,1
6,남가좌2동,1
7,논현2동,1
8,대흥동,1
9,도화동,1


In [9]:
table_z = pd.read_excel("fin_dataset_zero.xlsx")

In [12]:
len(table_z[table_z["count_univ"]!=0])

53

In [7]:
table_z["대학교개수"]=""

for i in range(424):
    for j in range(53):
        if table_z["dong_nm"][i] == univ["행정동"][j]:
            table_z["대학교개수"][i] = univ["학교개수"][j]
        else: 
            table_z["대학교개수"][i] = 0

C:\Users\ajm10\AppData\Local\Temp\ipykernel_1344\3464840686.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table_z["대학교개수"][i] = 0
C:\Users\ajm10\AppData\Local\Temp\ipykernel_1344\3464840686.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table_z["대학교개수"][i] = univ["학교개수"][j]


In [8]:
table_z[table_z["dong_nm"]=="화양동"]

,goo_nm,dong_ID,dong_nm,dong_area,sum_pop,one_pop,cc_value,count_zero,count_market,count_univ,count_bus,count_station,sum_bus,sum_station,sum_order,sum_waste,y,x,대학교개수
415,광진구,11215710,화양동,1.16,16621,11032,30,0,1,0,21,1,3561804,13527276,62301,1584.9,37.54325,127.073393,0


In [12]:
table_z

,goo_nm,dong_ID,dong_nm,dong_area,sum_pop,one_pop,cc_value,count_zero,count_market,count_univ,count_bus,count_station,sum_bus,sum_station,sum_order,sum_waste,y,x,대학교개수
0,송파구,11710631,가락1동,1.46,9534,1045,0,0,1,0,10,2,2605376,6434890,73694,4271.9,37.496537,127.105726,0
1,송파구,11710632,가락2동,1.34,11482,1892,10,0,2,0,22,2,2181971,3449427,73694,4271.9,37.498639,127.126488,0
2,송파구,11710620,가락본동,0.96,10576,2642,30,1,1,0,25,0,3081377,0,73694,4271.9,37.495585,127.121830,0
3,구로구,11530595,가리봉동,0.40,4901,2244,10,0,0,0,10,0,2235038,0,50971,1278.5,37.482704,126.888019,0
4,금천구,11545510,가산동,2.52,14124,8016,30,0,6,0,105,2,10824582,14509425,42334,3794.8,37.476681,126.883767,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,용산구,11170580,효창동,3.01,3922,894,0,0,0,0,14,1,1258577,1545629,36881,1052.6,37.542828,126.960951,0
420,용산구,11170510,후암동,0.44,7283,2314,20,0,0,0,19,0,1910323,0,36881,1052.6,37.549901,126.980613,0
421,동대문구,11230720,휘경1동,0.76,6643,2730,10,0,1,0,15,1,3743167,9501588,60925,3237.0,37.574428,127.039712,0
422,동대문구,11230730,휘경2동,0.63,10915,3915,29,0,0,0,17,0,3812785,0,60925,3237.0,37.574428,127.039712,0


In [ ]:
table_z.to_excel("fin_dataset_zero2.xlsx")